In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
import matplotlib.gridspec as gridspec

In [2]:
%%time
train_metadata_kaggle = dd.read_csv('klm_train.csv')
test_metadata_kaggle = dd.read_csv('klm_test.csv')
train_metadata_kaggle = train_metadata_kaggle.compute()
test_metadata_kaggle = test_metadata_kaggle.compute()
print(train_metadata_kaggle.shape,test_metadata_kaggle.shape)

(7848, 64) (3492890, 63)
CPU times: user 1min 39s, sys: 6.08 s, total: 1min 45s
Wall time: 18.5 s


In [3]:
#Always seed the randomness of this universe
np.random.seed(51)

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
%%time
train_metadata = dd.read_csv('train_metadata_final.csv')
test_metadata = dd.read_csv('test_metadata_final.csv')
train_metadata = train_metadata.compute()
test_metadata = test_metadata.compute()
print(train_metadata.shape,test_metadata.shape)

(7848, 233) (3492890, 232)
CPU times: user 3min 39s, sys: 12.2 s, total: 3min 51s
Wall time: 42.5 s


In [6]:
deleted_list = ['ra','decl','gal_l','gal_b','hostgal_specz','hostgal_photoz','hostgal_photoz_err','distmod','mwebv','target']

In [7]:
train_metadata.drop(deleted_list,axis=1,inplace=True)
test_metadata.drop(deleted_list[:-1],axis=1,inplace=True)

In [8]:
print(train_metadata.shape,test_metadata.shape)

(7848, 223) (3492890, 223)


In [15]:
train_metadata_kaggle.head()

,object_id,flux_min,flux_max,flux_mean,flux_median,flux_std,flux_skew,flux_err_min,flux_err_max,flux_err_mean,flux_err_median,flux_err_std,flux_err_skew,detected_mean,flux_ratio_sq_sum,flux_ratio_sq_skew,flux_by_flux_ratio_sq_sum,flux_by_flux_ratio_sq_skew,flux_w_mean,flux_diff1,flux_diff2,flux_diff3,"0__fft_coefficient__coeff_0__attr_""abs""","0__fft_coefficient__coeff_1__attr_""abs""",0__kurtosis,0__skewness,"1__fft_coefficient__coeff_0__attr_""abs""","1__fft_coefficient__coeff_1__attr_""abs""",1__kurtosis,1__skewness,"2__fft_coefficient__coeff_0__attr_""abs""","2__fft_coefficient__coeff_1__attr_""abs""",2__kurtosis,2__skewness,"3__fft_coefficient__coeff_0__attr_""abs""","3__fft_coefficient__coeff_1__attr_""abs""",3__kurtosis,3__skewness,"4__fft_coefficient__coeff_0__attr_""abs""","4__fft_coefficient__coeff_1__attr_""abs""",4__kurtosis,4__skewness,"5__fft_coefficient__coeff_0__attr_""abs""","5__fft_coefficient__coeff_1__attr_""abs""",5__kurtosis,5__skewness,flux__length,flux__longest_strike_above_mean,flux__longest_strike_below_mean,flux__mean_abs_change,flux__mean_change,flux_by_flux_ratio_sq__longest_strike_above_mean,flux_by_flux_ratio_sq__longest_strike_below_mean,mjd__mean_abs_change,mjd__mean_change,mjd_diff_det,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target,haversine,latlon1,hostgal_photoz_certain
0,615,-1100.440063,660.626343,-123.096998,-89.477524,394.109851,-0.349540,2.130510,12.845472,4.482743,3.835268,1.744747,1.623740,0.946023,2.929669e+06,0.812722,-9.601766e+08,-1.414322,-327.742307,1761.066406,-14.306331,-5.373326,205.036926,1628.427737,-1.475181,0.128917,22370.594834,2806.374162,-1.255123,0.415580,7780.500807,2805.598113,-1.409885,0.339918,7024.003068,2536.068846,-1.449858,0.293128,3245.366349,2741.539785,-1.548319,0.200096,2704.641265,2893.344217,-1.592820,0.125268,352.0,19.0,29.0,202.114067,1.999688,35.0,4.0,2.631898,2.631898,873.7903,0.0000,0.0000,NaN,0.017,92,0.319006,-1.528827,0.000000
1,713,-14.735178,14.770886,-1.423351,-0.873033,6.471144,0.014989,0.639458,9.115748,2.359620,1.998217,1.509888,1.633246,0.171429,5.886068e+03,3.439423,-2.875087e+04,-3.454554,-4.884564,29.506064,-20.730002,-6.040676,190.427851,299.586559,-1.014003,0.260052,57.109047,192.539229,-1.097170,-0.087865,44.477327,191.057528,-1.188472,-0.022678,55.270113,212.522263,-1.142896,-0.167176,50.414646,203.892482,-1.190245,-0.064134,100.473776,143.963093,-0.797047,0.218182,350.0,50.0,73.0,2.935177,-0.050944,199.0,8.0,14.352571,14.352571,846.8017,1.6267,0.2552,45.4063,0.007,88,1.698939,3.258921,2.099614
2,730,-19.159811,47.310059,2.267434,0.409172,8.022239,3.177854,0.695106,11.281384,2.471061,1.990851,1.721134,1.823726,0.069697,4.124452e+03,5.480405,1.046502e+05,5.989138,25.373110,66.469870,29.315018,2.619697,3.461790,4.729538,0.474215,0.356910,7.334944,13.515895,0.976374,0.471342,124.845250,119.500254,5.131290,2.385066,168.280524,162.799417,7.125665,2.662075,219.745132,202.532898,6.081065,2.537802,231.509177,199.286370,3.583130,1.680352,330.0,13.0,32.0,4.227614,-0.008131,4.0,222.0,3.580623,3.580623,78.7737,0.2262,0.0157,40.2561,0.021,42,1.818030,3.128522,0.229779
3,745,-15.494463,220.795212,8.909206,1.035895,27.558208,4.979826,0.567170,55.892746,2.555576,1.819875,3.537324,10.741655,0.173789,9.416165e+04,9.611274,1.439125e+07,11.141069,152.835617,236.289675,26.521968,1.546038,129.421659,123.298327,4.629801,2.023211,320.174052,280.440312,50.868880,7.007099,543.845781,491.548270,36.088137,5.688194,807.123762,710.721942,16.392533,3.751603,735.528417,680.055280,13.747434,3.476420,591.037583,523.503586,12.134629,3.170857,351.0,19.0,115.0,7.065548,0.008044,4.0,201.0,2.061453,2.061453,123.6872,0.2813,1.1523,40.7951,0.007,90,0.495223,6.893743,0.890445
4,1124,-16.543753,143.600189,7.145702,1.141288,20.051722,4.406298,0.695277,11.383690,2.753004,2.214854,1.933837,1.794938,0.173295,3.432418e+04,7.868462,3.015599e+06,7.908174,87.856390,160.143942,22.411225,1.822792,41.639721,32.987125,0.822496,-0.332169,268.808929,207.812015,6.112295,2.377222,5

In [9]:
train_metadata.head()

,object_id,ddf,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,percent_p5_region_plus_2,percent_p5_region_plus_3,percent_p5_region_plus_4
0,615,1,2

In [10]:
test_id = test_metadata_kaggle['object_id']

In [11]:
cesium_features = pd.read_csv('cesium_features.csv')
my_features = pd.read_csv('my_features.csv')

In [12]:
def multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds
    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return loss


def lgb_multi_weighted_logloss(y_true, y_preds):
    """
    @author olivier https://www.kaggle.com/ogrellier
    multi logloss for PLAsTiCC challenge
    """
    # class_weights taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    if len(np.unique(y_true)) > 14:
        classes.append(99)
        class_weight[99] = 2
    y_p = y_preds.reshape(y_true.shape[0], len(classes), order='F')

    # Trasform y_true in dummies
    y_ohe = pd.get_dummies(y_true)
    # Normalize rows and limit y_preds to 1e-15, 1-1e-15
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    # Transform to log
    y_p_log = np.log(y_p)
    # Get the log for ones, .values is used to drop the index of DataFrames
    # Exclude class 99 for now, since there is no class99 in the training set
    # we gave a special process for that class
    y_log_ones = np.sum(y_ohe.values * y_p_log, axis=0)
    # Get the number of positives for each class
    nb_pos = y_ohe.sum(axis=0).values.astype(float)
    # Weight average and divide by the number of positives
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    y_w = y_log_ones * class_arr / nb_pos

    loss = - np.sum(y_w) / np.sum(class_arr)
    return 'wloss', loss, False

In [13]:
my_features['fold_sum'].value_counts()

1    122
2     78
3     15
0      7
Name: fold_sum, dtype: int64

In [96]:
columns_0 = []
columns_0.append('A0_sum_flux')
columns_0.append('A0_mean_flux')
columns_0.append('A0_std_detected')
columns_0.append('A1_mean_detected')
columns_0.append('A2_sum_detected')
columns_0.append('A4_mean_detected')
columns_0.append('A5_std_detected')
columns_0.append('A5_mean_detected')
columns_0.append('percent_p2_region_minus_1')

In [97]:
columns_added = list(my_features[my_features['fold_sum'] == 0]['column_name'])

In [98]:
columns_added

['A0_sum_flux',
 'A0_mean_flux',
 'A1_min_flux',
 'A2_min_flux',
 'A5_std_detected',
 'A5_sum_detected',
 'A5_mean_detected']

In [99]:
columns_added = [x for x in columns_added if x not in columns_0 ]

In [100]:
columns_added

['A1_min_flux', 'A2_min_flux', 'A5_sum_detected']

In [101]:
columns_added = ['A2_min_flux','A5_sum_detected']

In [102]:
columns_0 = columns_0 + columns_added

In [103]:
print(columns_0)

['A0_sum_flux', 'A0_mean_flux', 'A0_std_detected', 'A1_mean_detected', 'A2_sum_detected', 'A4_mean_detected', 'A5_std_detected', 'A5_mean_detected', 'percent_p2_region_minus_1', 'A2_min_flux', 'A5_sum_detected']


In [105]:
columns_0 = ['A0_sum_flux', 'A0_mean_flux', 'A0_std_detected', 'A1_mean_detected', 'A2_sum_detected', 'A4_mean_detected', 'A5_std_detected', 'A5_mean_detected', 'percent_p2_region_minus_1', 'A2_min_flux', 'A5_sum_detected']

In [106]:
test_metadata.head()

,object_id,ddf,NG_min_flux_err,NG_max_flux_err,NG_std_flux_err,NG_sum_flux_err,NG_mean_flux_err,NG_median_flux_err,NG_min_flux,NG_max_flux,NG_std_flux,NG_sum_flux,NG_mean_flux,NG_median_flux,NG_count_detected,NG_std_detected,NG_sum_detected,NG_mean_detected,NG_median_detected,A0_min_flux_err,A0_max_flux_err,A0_std_flux_err,A0_sum_flux_err,A0_mean_flux_err,A0_median_flux_err,A0_min_flux,A0_max_flux,A0_std_flux,A0_sum_flux,A0_mean_flux,A0_median_flux,A0_count_detected,A0_std_detected,A0_sum_detected,A0_mean_detected,A0_median_detected,A1_min_flux_err,A1_max_flux_err,A1_std_flux_err,A1_sum_flux_err,A1_mean_flux_err,A1_median_flux_err,A1_min_flux,A1_max_flux,A1_std_flux,A1_sum_flux,A1_mean_flux,A1_median_flux,A1_count_detected,A1_std_detected,A1_sum_detected,A1_mean_detected,A1_median_detected,A2_min_flux_err,A2_max_flux_err,A2_std_flux_err,A2_sum_flux_err,A2_mean_flux_err,A2_median_flux_err,A2_min_flux,A2_max_flux,A2_std_flux,A2_sum_flux,A2_mean_flux,A2_median_flux,A2_count_detected,A2_std_detected,A2_sum_detected,A2_mean_detected,A2_median_detected,A3_min_flux_err,A3_max_flux_err,A3_std_flux_err,A3_sum_flux_err,A3_mean_flux_err,A3_median_flux_err,A3_min_flux,A3_max_flux,A3_std_flux,A3_sum_flux,A3_mean_flux,A3_median_flux,A3_count_detected,A3_std_detected,A3_sum_detected,A3_mean_detected,A3_median_detected,A4_min_flux_err,A4_max_flux_err,A4_std_flux_err,A4_sum_flux_err,A4_mean_flux_err,A4_median_flux_err,A4_min_flux,A4_max_flux,A4_std_flux,A4_sum_flux,A4_mean_flux,A4_median_flux,A4_count_detected,A4_std_detected,A4_sum_detected,A4_mean_detected,A4_median_detected,A5_min_flux_err,A5_max_flux_err,A5_std_flux_err,A5_sum_flux_err,A5_mean_flux_err,A5_median_flux_err,A5_min_flux,A5_max_flux,A5_std_flux,A5_sum_flux,A5_mean_flux,A5_median_flux,A5_count_detected,A5_std_detected,A5_sum_detected,A5_mean_detected,A5_median_detected,p0_region_minus_4,p0_region_minus_3,p0_region_minus_2,p0_region_minus_1,p0_region_plus_1,p0_region_plus_2,p0_region_plus_3,p0_region_plus_4,p1_region_minus_4,p1_region_minus_3,p1_region_minus_2,p1_region_minus_1,p1_region_plus_1,p1_region_plus_2,p1_region_plus_3,p1_region_plus_4,p2_region_minus_4,p2_region_minus_3,p2_region_minus_2,p2_region_minus_1,p2_region_plus_1,p2_region_plus_2,p2_region_plus_3,p2_region_plus_4,p3_region_minus_4,p3_region_minus_3,p3_region_minus_2,p3_region_minus_1,p3_region_plus_1,p3_region_plus_2,p3_region_plus_3,p3_region_plus_4,p4_region_minus_4,p4_region_minus_3,p4_region_minus_2,p4_region_minus_1,p4_region_plus_1,p4_region_plus_2,p4_region_plus_3,p4_region_plus_4,p5_region_minus_4,p5_region_minus_3,p5_region_minus_2,p5_region_minus_1,p5_region_plus_1,p5_region_plus_2,p5_region_plus_3,p5_region_plus_4,number_of_0s,number_of_1s,number_of_2s,number_of_3s,number_of_4s,number_of_5s,percent_p0_region_minus_4,percent_p0_region_minus_3,percent_p0_region_minus_2,percent_p0_region_minus_1,percent_p0_region_plus_1,percent_p0_region_plus_2,percent_p0_region_plus_3,percent_p0_region_plus_4,percent_p1_region_minus_4,percent_p1_region_minus_3,percent_p1_region_minus_2,percent_p1_region_minus_1,percent_p1_region_plus_1,percent_p1_region_plus_2,percent_p1_region_plus_3,percent_p1_region_plus_4,percent_p2_region_minus_4,percent_p2_region_minus_3,percent_p2_region_minus_2,percent_p2_region_minus_1,percent_p2_region_plus_1,percent_p2_region_plus_2,percent_p2_region_plus_3,percent_p2_region_plus_4,percent_p3_region_minus_4,percent_p3_region_minus_3,percent_p3_region_minus_2,percent_p3_region_minus_1,percent_p3_region_plus_1,percent_p3_region_plus_2,percent_p3_region_plus_3,percent_p3_region_plus_4,percent_p4_region_minus_4,percent_p4_region_minus_3,percent_p4_region_minus_2,percent_p4_region_minus_1,percent_p4_region_plus_1,percent_p4_region_plus_2,percent_p4_region_plus_3,percent_p4_region_plus_4,percent_p5_region_minus_4,percent_p5_region_minus_3,percent_p5_region_minus_2,percent_p5_region_minus_1,percent_p5_region_plus_1,percent_p5_region_plus_2,percent_p5_region_plus_3,percent_p5_region_plus_4
0,13,1,0.

In [107]:
cesium_imps = ['object_id','target','flux_max','flux_mean','flux_median','flux_skew','flux_err_min',
               'flux_err_mean','detected_mean','flux_ratio_sq_skew',
               'flux_by_flux_ratio_sq_skew','flux_w_mean','flux_diff1',
              'flux_diff2','flux_diff3','0__fft_coefficient__coeff_0__attr_"abs"',
               '0__fft_coefficient__coeff_1__attr_"abs"','0__skewness',
               '1__fft_coefficient__coeff_0__attr_"abs"','1__fft_coefficient__coeff_1__attr_"abs"',
               '1__kurtosis','1__skewness','2__fft_coefficient__coeff_0__attr_"abs"',
               '2__fft_coefficient__coeff_1__attr_"abs"','2__kurtosis','2__skewness',
               '3__fft_coefficient__coeff_0__attr_"abs"',
               '3__fft_coefficient__coeff_1__attr_"abs"',
               '3__kurtosis','3__skewness',
               '4__fft_coefficient__coeff_0__attr_"abs"',
               '4__fft_coefficient__coeff_1__attr_"abs"',
               '4__kurtosis','4__skewness',
               '5__fft_coefficient__coeff_0__attr_"abs"',
               '5__fft_coefficient__coeff_1__attr_"abs"',
               '5__kurtosis','5__skewness',
               'flux__longest_strike_above_mean',
               'flux__longest_strike_below_mean',
               'flux_mean_abs_change',
               'flux_by_flux_ratio_sq__longest_strike_above_mean',
               'flux_by_flux_ratio_sq__longest_strike_below_mean',
               'mjd__mean_abs_change','mjd__mean_change',
               'mjd_diff_det','hostgal_photoz','hostgal_photoz_err',
               'distmod','hostgal_photoz_certain',
               'NG_min_flux_err','NG_max_flux','NG_median_flux',
               'NG_sum_detected','NG_mean_detected','A0_max_flux'
               'A0_std_flux','A0_sum_flux','A0_mean_flux',
               'A0_median_flux','A0_std_detected','A0_sum_detected',
               'A0_mean_detected','A1_min_flux_err','A1_max_flux_err',
               'A1_sum_flux_err','A1_mean_flux_err','A1_median_flux_err',
               'A1_min_flux','A1_max_flux','A1_std_flux','A1_mean_flux',
               'A1_median_flux','A1_mean_detected','A1_median_detected',
               'A2_min_flux_err','A2_min_flux','A2_max_flux',
               'A2_std_flux','A2_median_flux','A2_std_detected',
               'A2_mean_detected','A3_sum_detected','A3_mean_detected',
               'A4_mean_flux','A4_sum_detected','A4_mean_detected',
               'A5_max_flux','A5_std_flux','A5_sum_flux','A5_mean_flux',
               'A5_std_detected','A5_mean_detected','A5_median_detected',
               'p0_region_plus_1','p3_region_plus_4'
               'percent_p0_region_minus_1','percent_p2_region_minus_2',
               'percent_p2_region_minus_1','percent_p3_region_minus_1',
               'percent_p3_region_plus_4','percent_p4_region_plus_4',
               'percent_p5_region_minus_1'
              ]

In [108]:
cesium_imps2 = [x for x in cesium_imps if x not in columns_0 + ['object_id','target']]

In [109]:
cesium_imps2 = [x for x in cesium_imps2 if x in train_metadata.columns]

In [112]:
print(cesium_imps2)

['NG_min_flux_err', 'NG_max_flux', 'NG_median_flux', 'NG_sum_detected', 'NG_mean_detected', 'A0_median_flux', 'A0_sum_detected', 'A0_mean_detected', 'A1_min_flux_err', 'A1_max_flux_err', 'A1_sum_flux_err', 'A1_mean_flux_err', 'A1_median_flux_err', 'A1_min_flux', 'A1_max_flux', 'A1_std_flux', 'A1_mean_flux', 'A1_median_flux', 'A1_median_detected', 'A2_min_flux_err', 'A2_max_flux', 'A2_std_flux', 'A2_median_flux', 'A2_std_detected', 'A2_mean_detected', 'A3_sum_detected', 'A3_mean_detected', 'A4_mean_flux', 'A4_sum_detected', 'A5_max_flux', 'A5_std_flux', 'A5_sum_flux', 'A5_mean_flux', 'A5_median_detected', 'p0_region_plus_1', 'percent_p2_region_minus_2', 'percent_p3_region_minus_1', 'percent_p3_region_plus_4', 'percent_p4_region_plus_4', 'percent_p5_region_minus_1']


In [113]:
train_metadata[['object_id'] + columns_0 ].head()

,object_id,A0_sum_flux,A0_mean_flux,A0_std_detected,A1_mean_detected,A2_sum_detected,A4_mean_detected,A5_std_detected,A5_mean_detected,percent_p2_region_minus_1,A2_min_flux,A5_sum_detected
0,615,-205.03693,-3.254554,0.3528,0.9653,57,0.9830,0.2854,0.9120,0.362,-682.000,52
1,713,-190.42786,-2.720398,0.3525,0.2678,15,0.0893,0.0000,0.0000,0.250,-10.070,0
2,730,-3.46179,-0.048080,0.0000,0.0000,7,0.0980,0.2715,0.0784,0.769,-2.850,4
3,745,129.42166,1.797523,0.1655,0.1250,16,0.2322,0.3364,0.1273,0.768,-2.160,7
4,1124,41.63972,0.660948,0.0000,0.2241,18,0.1724,0.2578,0.0702,0.707,-2.084,4


In [117]:
temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + columns_0 ],on = 'object_id',how = 'left')

In [118]:
%%time
final_dict = {}

loss_list = []
#temp = train_metadata_kaggle.copy()
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 500, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.1, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

0 0.6526784960262693
1 0.617515730282096
2 0.6776882193363233
3 0.6232161919670319
4 0.6085317600247506
MULTI WEIGHTED LOG LOSS : 0.63612 
CPU times: user 7min 53s, sys: 954 ms, total: 7min 54s
Wall time: 1min 3s


In [120]:
print(columns_0)

['A0_sum_flux', 'A0_mean_flux', 'A0_std_detected', 'A1_mean_detected', 'A2_sum_detected', 'A4_mean_detected', 'A5_std_detected', 'A5_mean_detected', 'percent_p2_region_minus_1', 'A2_min_flux', 'A5_sum_detected']


In [121]:
print(cesium_imps2)

['NG_min_flux_err', 'NG_max_flux', 'NG_median_flux', 'NG_sum_detected', 'NG_mean_detected', 'A0_median_flux', 'A0_sum_detected', 'A0_mean_detected', 'A1_min_flux_err', 'A1_max_flux_err', 'A1_sum_flux_err', 'A1_mean_flux_err', 'A1_median_flux_err', 'A1_min_flux', 'A1_max_flux', 'A1_std_flux', 'A1_mean_flux', 'A1_median_flux', 'A1_median_detected', 'A2_min_flux_err', 'A2_max_flux', 'A2_std_flux', 'A2_median_flux', 'A2_std_detected', 'A2_mean_detected', 'A3_sum_detected', 'A3_mean_detected', 'A4_mean_flux', 'A4_sum_detected', 'A5_max_flux', 'A5_std_flux', 'A5_sum_flux', 'A5_mean_flux', 'A5_median_detected', 'p0_region_plus_1', 'percent_p2_region_minus_2', 'percent_p3_region_minus_1', 'percent_p3_region_plus_4', 'percent_p4_region_plus_4', 'percent_p5_region_minus_1']


In [ ]:
['A3_std_flux','A1_min_flux_err','A3_sum_flux','A1_std_detected',
 'A1_sum_detected',
 'A1_mean_detected','percent_p0_region_minus_1',
'A0_std_detected','A4_mean_detected','A0_mean_flux',]

In [116]:
%%time
final_dict = {}
for column_ in cesium_imps2:
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id'] + columns_0 + [column_] ],on = 'object_id',how = 'left')
    

    loss_list = []
    #temp = train_metadata_kaggle.copy()
    #temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    # Taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    #print('Unique classes : ', classes)

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 500, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10, 
    'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    'reg_alpha': 0.1, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list

NG_min_flux_err MULTI WEIGHTED LOG LOSS : 0.63896 
NG_max_flux MULTI WEIGHTED LOG LOSS : 0.64041 
NG_median_flux MULTI WEIGHTED LOG LOSS : 0.64041 
NG_sum_detected MULTI WEIGHTED LOG LOSS : 0.64056 
NG_mean_detected MULTI WEIGHTED LOG LOSS : 0.64091 
A0_median_flux MULTI WEIGHTED LOG LOSS : 0.63948 
A0_sum_detected MULTI WEIGHTED LOG LOSS : 0.63978 
A0_mean_detected MULTI WEIGHTED LOG LOSS : 0.64014 
A1_min_flux_err MULTI WEIGHTED LOG LOSS : 0.63936 
A1_max_flux_err MULTI WEIGHTED LOG LOSS : 0.64084 
A1_sum_flux_err MULTI WEIGHTED LOG LOSS : 0.64229 
A1_mean_flux_err MULTI WEIGHTED LOG LOSS : 0.64105 
A1_median_flux_err MULTI WEIGHTED LOG LOSS : 0.63903 
A1_min_flux MULTI WEIGHTED LOG LOSS : 0.64125 
A1_max_flux MULTI WEIGHTED LOG LOSS : 0.64042 
A1_std_flux MULTI WEIGHTED LOG LOSS : 0.63843 
A1_mean_flux MULTI WEIGHTED LOG LOSS : 0.64033 
A1_median_flux MULTI WEIGHTED LOG LOSS : 0.64116 
A1_median_detected MULTI WEIGHTED LOG LOSS : 0.64053 
A2_min_flux_err MULTI WEIGHTED LOG LOSS : 0.

In [122]:
cesium_imp2_2 = ['A3_std_flux','A1_min_flux_err','A3_sum_flux','A1_std_detected','A1_sum_detected','A1_mean_detected','percent_p0_region_minus_1',]

In [123]:
%%time
final_dict = {}
for column_ in cesium_imp2_2:
    temp = train_metadata_kaggle.copy()
    temp = temp.merge(train_metadata[['object_id'] + columns_0 + [column_] ],on = 'object_id',how = 'left')
    

    loss_list = []
    #temp = train_metadata_kaggle.copy()
    #temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')
    y = temp['target']
    del temp['target']
    classes = sorted(y.unique())

    # Taken from Giba's topic : https://www.kaggle.com/titericz
    # https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
    # with Kyle Boone's post https://www.kaggle.com/kyleboone
    class_weight = {
        c: 1 for c in classes
    }
    for c in [64, 15]:
        class_weight[c] = 2

    #print('Unique classes : ', classes)

    train_id = temp['object_id']
    del temp['object_id']
    # Compute weights
    w = y.value_counts()
    weights = {i : np.sum(w) / w[i] for i in w.index}
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
    clfs = []
    importances = pd.DataFrame()
    lgb_params = {
    'random_state':51,
    'device': 'cpu', 
    'objective': 'multiclass', 
    'num_class': 14, 
    'boosting_type': 'gbdt', 
    'n_jobs': -1, 
    'max_depth': 7, 
    'n_estimators': 500, 
    'subsample_freq': 2, 
    'subsample_for_bin': 5000, 
    'min_data_per_group': 100, 
    'max_cat_to_onehot': 4, 
    'cat_l2': 1.0, 
    'cat_smooth': 59.5, 
    'max_cat_threshold': 32, 
    'metric_freq': 10, 
    'verbosity': -1, 
    'metric': 'multi_logloss', 
    'xgboost_dart_mode': False, 
    'uniform_drop': False, 
    'colsample_bytree': 0.5, 
    'drop_rate': 0.173, 
    'learning_rate': 0.0267, 
    'max_drop': 5, 
    'min_child_samples': 10, 
    'min_child_weight': 100.0, 
    'min_split_gain': 0.1, 
    'num_leaves': 7, 
    'reg_alpha': 0.1, 
    'reg_lambda': 0.00023, 
    'skip_drop': 0.44, 
    'subsample': 0.75}
    oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
    for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
        trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
        val_x, val_y = temp.iloc[val_], y.iloc[val_]

        clf = lgb.LGBMClassifier(**lgb_params)
        clf.fit(
            trn_x, trn_y,
            eval_set=[(trn_x, trn_y), (val_x, val_y)],
            eval_metric=lgb_multi_weighted_logloss,
            verbose=False,
            early_stopping_rounds=50,
            sample_weight=trn_y.map(weights)
        )
        oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
        loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
        loss_list.append(loss_oof)
        #print(fold_,loss_oof)

        imp_df = pd.DataFrame()
        imp_df['feature'] = temp.columns
        imp_df['gain'] = clf.feature_importances_
        imp_df['fold'] = fold_ + 1
        importances = pd.concat([importances, imp_df], axis=0, sort=False)

        clfs.append(clf)
    print(column_,'MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
    final_dict[column_] = loss_list

A3_std_flux MULTI WEIGHTED LOG LOSS : 0.64107 
A1_min_flux_err MULTI WEIGHTED LOG LOSS : 0.63936 
A3_sum_flux MULTI WEIGHTED LOG LOSS : 0.63988 
A1_std_detected MULTI WEIGHTED LOG LOSS : 0.64045 
A1_sum_detected MULTI WEIGHTED LOG LOSS : 0.64010 
A1_mean_detected MULTI WEIGHTED LOG LOSS : 0.63904 
percent_p0_region_minus_1 MULTI WEIGHTED LOG LOSS : 0.64100 
CPU times: user 54min 27s, sys: 6.51 s, total: 54min 34s
Wall time: 7min 14s


In [ ]:
final_dict2 = pd.Series(final_dict)

In [ ]:
final_dict2 = pd.DataFrame(final_dict2)

In [ ]:
final_dict2['fold1'] = final_dict2[0].apply(lambda x: x[0])
final_dict2['fold2'] = final_dict2[0].apply(lambda x: x[1])
final_dict2['fold3'] = final_dict2[0].apply(lambda x: x[2])
final_dict2['fold4'] = final_dict2[0].apply(lambda x: x[3])
final_dict2['fold5'] = final_dict2[0].apply(lambda x: x[4])

In [ ]:
del final_dict2[0]

In [ ]:
final_dict2 = final_dict2.reset_index(drop=False)

In [ ]:
final_dict2 = final_dict2.rename(columns={'index':'column_name'})

In [ ]:
final_dict2['fold1_1'] = (final_dict2['fold1'] > 0.66215) * 1
final_dict2['fold2_1'] = (final_dict2['fold2'] > 0.61915) * 1
final_dict2['fold3_1'] = (final_dict2['fold3'] > 0.67933) * 1
final_dict2['fold4_1'] = (final_dict2['fold4'] > 0.61120) * 1
final_dict2['fold5_1'] = (final_dict2['fold5'] > 0.61399) * 1
final_dict2['fold_sum'] = final_dict2['fold1_1']+ final_dict2['fold2_1'] + final_dict2['fold3_1'] + final_dict2['fold4_1'] + final_dict2['fold5_1']

In [ ]:
final_dict2.head()

In [ ]:
final_dict2.to_csv('my_features_2.csv',index=False)

In [ ]:
final_dict2[final_dict2['fold_sum'] == 0]

# GOOD FEATURES

In [ ]:
good_features = ['__freq_varrat___5_',
                 '__freq_varrat___4_',
                 '__qso_log_chi2_qsonu___3_',
                '__qso_log_chi2_qsonu___1_',
                 '__qso_log_chi2_qsonu___5_',
                '__std___4_',
                 '__freq_varrat___3_',
                 '__amplitude___2_'
                ]

In [ ]:
#good_features = ['__freq_varrat___4_','__freq_varrat___5_']

In [ ]:
%%time
final_dict = {}

loss_list = []
#temp = train_metadata_kaggle.copy()
#temp = temp.merge(train_metadata[['object_id',column_]],on = 'object_id',how = 'left')

temp = train_metadata_kaggle.copy()
temp = temp.merge(train_metadata[['object_id'] + columns_0 + good_features  ],on = 'object_id',how = 'left')
y = temp['target']
del temp['target']
classes = sorted(y.unique())

print(temp.shape)

# Taken from Giba's topic : https://www.kaggle.com/titericz
# https://www.kaggle.com/c/PLAsTiCC-2018/discussion/67194
# with Kyle Boone's post https://www.kaggle.com/kyleboone
class_weight = {
    c: 1 for c in classes
}
for c in [64, 15]:
    class_weight[c] = 2

#print('Unique classes : ', classes)

train_id = temp['object_id']
del temp['object_id']
# Compute weights
w = y.value_counts()
weights = {i : np.sum(w) / w[i] for i in w.index}
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=51)
clfs = []
importances = pd.DataFrame()
lgb_params = {
'random_state':51,
'device': 'cpu', 
'objective': 'multiclass', 
'num_class': 14, 
'boosting_type': 'gbdt', 
'n_jobs': -1, 
'max_depth': 7, 
'n_estimators': 500, 
'subsample_freq': 2, 
'subsample_for_bin': 5000, 
'min_data_per_group': 100, 
'max_cat_to_onehot': 4, 
'cat_l2': 1.0, 
'cat_smooth': 59.5, 
'max_cat_threshold': 32, 
'metric_freq': 10, 
'verbosity': -1, 
'metric': 'multi_logloss', 
'xgboost_dart_mode': False, 
'uniform_drop': False, 
'colsample_bytree': 0.5, 
'drop_rate': 0.173, 
'learning_rate': 0.0267, 
'max_drop': 5, 
'min_child_samples': 10, 
'min_child_weight': 100.0, 
'min_split_gain': 0.1, 
'num_leaves': 7, 
'reg_alpha': 0.1, 
'reg_lambda': 0.00023, 
'skip_drop': 0.44, 
'subsample': 0.75}
oof_preds = np.zeros((len(temp), np.unique(y).shape[0]))
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    trn_x, trn_y = temp.iloc[trn_], y.iloc[trn_]
    val_x, val_y = temp.iloc[val_], y.iloc[val_]

    clf = lgb.LGBMClassifier(**lgb_params)
    clf.fit(
        trn_x, trn_y,
        eval_set=[(trn_x, trn_y), (val_x, val_y)],
        eval_metric=lgb_multi_weighted_logloss,
        verbose=False,
        early_stopping_rounds=50,
        sample_weight=trn_y.map(weights)
    )
    oof_preds[val_, :] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)
    loss_oof = multi_weighted_logloss(val_y, oof_preds[val_, :])
    #loss_list.append(loss_oof)
    print(fold_,loss_oof)

    imp_df = pd.DataFrame()
    imp_df['feature'] = temp.columns
    imp_df['gain'] = clf.feature_importances_
    imp_df['fold'] = fold_ + 1
    importances = pd.concat([importances, imp_df], axis=0, sort=False)

    clfs.append(clf)
print('MULTI WEIGHTED LOG LOSS : %.5f ' % multi_weighted_logloss(y_true=y, y_preds=oof_preds))
#final_dict[column_] = loss_list

In [ ]:
#temp = temp[cesium_imps2]